<a href="https://colab.research.google.com/github/nannthd/project_AIEngineer/blob/main/vector_drug_ResNet50%26YOLO%26Qdrant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/drug/data 50 class add_augment3.zip'

##image2vector qdrant

In [ ]:
!pip install tensorflow

In [ ]:
!pip install qdrant-client

In [ ]:
import os
import numpy as np
import uuid
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct
from qdrant_client.http.models import VectorParams, CollectionParams

# ใช้โมเดล ResNet50 ของ TensorFlow ในการทำ embedding
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

# เชื่อมต่อกับ Qdrant
qdrant_client = QdrantClient(
    url="https://8366dca9-9b40-481c-9a7c-102b62b118c2.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key="Ft3t6L99uoTsJW8IOk5VH2byKU-BQYqiieuxXFBDp99wo75od0ddAw",
)

def create_collection(collection_name, vector_size, distance_metric):
    try:
        # ตรวจสอบว่าคอลเลกชันมีอยู่แล้วหรือไม่
        collections = qdrant_client.get_collections().collections
        existing_collections = [col.name for col in collections]

        if collection_name in existing_collections:
            print(f"Collection {collection_name} already exists.")
        else:
            # สร้างคอลเลกชัน
            qdrant_client.create_collection(
                collection_name=collection_name,
                vectors_config=VectorParams(
                    size=vector_size,  # ขนาดของเวกเตอร์ embedding
                    distance=distance_metric  # ระยะทางที่ใช้ในการคำนวณ
                )
            )
            print(f"Collection {collection_name} created successfully.")
    except Exception as e:
        print(f"An error occurred while creating the collection: {e}")

def image_embedding(path):
    img = Image.open(path)
    img = img.resize((640, 640))  # ปรับขนาดภาพให้เข้ากับ ResNet50
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    return preds[0]

def process_images_in_folder(folder_path, collection_name):
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

    for idx, image_file in enumerate(image_files):
        image_path = os.path.join(folder_path, image_file)
        embedding = image_embedding(image_path)
        class_name = os.path.basename(folder_path)  # ใช้ชื่อโฟลเดอร์เป็นคลาส
        point_id = str(uuid.uuid4())  # ใช้ UUID เป็น ID
        point = PointStruct(id=point_id, vector=embedding.tolist(), payload={"class": class_name, "image_label": image_file})
        try:
            qdrant_client.upsert(collection_name=collection_name, points=[point])
            print(f"Uploaded {image_file} to collection {collection_name} with class {class_name}")
        except Exception as e:
            print(f"An error occurred while uploading {image_file}: {e}")

def process_all_folders(base_folder_path):
    collection_name = "vector_drug"  # ใช้ชื่อคอลเลกชันเดียวสำหรับเก็บข้อมูลทั้งหมด

    create_collection(collection_name, vector_size=2048, distance_metric='Cosine')

    subfolders = [f.path for f in os.scandir(base_folder_path) if f.is_dir()]

    for subfolder in subfolders:
        process_images_in_folder(subfolder, collection_name)
        print(f"Embedding data from {os.path.basename(subfolder)} has been uploaded to Qdrant")

# ตัวอย่างการใช้งาน
base_folder_path = '/content/data 50 class add_augment'
process_all_folders(base_folder_path)

##Test using YOLO & ResNet50

In [ ]:
!pip install ultralytics

In [ ]:
# Crop All
import os
from PIL import Image
from ultralytics import YOLO

# Load your YOLOv8 model
model = YOLO('/content/drive/MyDrive/drug/model.pt')

# Define the input path (can be a single image or a folder)
path = '/content/drive/MyDrive/drug/test'

# Define the crop directory
crop_dir = '/content//drive/MyDrive/drug/CorpTest'
os.makedirs(crop_dir, exist_ok=True)

# Function to get image ID from file name
def get_image_id(file_name):
    return os.path.splitext(os.path.basename(file_name))[0]

# Function to perform object detection and cropping
def detect_and_crop(image_path, image_id):
    # Predict with the YOLO model
    results = model(image_path)

    # Parse the results
    detected = False
    for result in results:
        # Extract boxes, labels, and scores
        boxes = result.boxes.xyxy  # x1, y1, x2, y2
        labels = result.boxes.cls  # class ids
        scores = result.boxes.conf  # confidence scores

        # Convert tensor to numpy for easier processing
        boxes = boxes.cpu().detach().numpy()
        labels = labels.cpu().detach().numpy()
        scores = scores.cpu().detach().numpy()

        # Threshold for detection confidence
        threshold = 0.3

        # List of target labels (Replace with actual indices of your classes in the YOLO model)
        target_labels = ['Amlopine_10mg', 'Amlopine_5mg', 'Anapril_5mg', 'Betalol-10-mg', 'Betalol-40-mg',
                         'Blopress-16-mg', 'Blopress-8-mg', 'Blopress-Plus-16-mg', 'Blopress-Plus-8-mg',
                         'Caduet-10_10-mg', 'Caduet-5_10-mg', 'Daonil-5-mg', 'Diamicron-MR-60-mg',
                         'Diovan-160-mg', 'Diovan-80-mg', 'Forxiga-10-mg', 'Galvus-50-mg', 'Galvus_Met_50_1000mg',
                         'Gliclazide_80mg', 'Gliparil-2-mg', 'Glucophage-500-mg', 'Glucophage-850-mg',
                         'Glucophage-XR-1000-mg', 'Glucophage-XR-750-mg', 'Glyxambi-25_5-mg', 'Janumet-50_1000-mg',
                         'Januvia-100-mg', 'Jardiance-10-mg', 'Jardiance-25-mg', 'Jardiance-Duo-12_5_1000-mg',
                         'Lanzaar-100-mg', 'Lercadip-20-mg', 'Madiplot-10-mg', 'Madiplot_20mg',
                         'Metoprolol_Stada_100mg', 'Micardis_40mg', 'Micardis_Plus_80_12_5mg', 'Minidiab_5mg',
                         'Novonorm_1mg', 'Novonorm_2mg', 'Oseni_25_15mg', 'Poli_uretic', 'Prenolol_100mg',
                         'Prenolol_25mg', 'Prenolol_50mg', 'Tanzaril_50mg', 'Utmos_15mg', 'Utmos_30mg',
                         'Xigduo_XR_10_1000mg', 'Zanidip_10mg']

        # Create a map from YOLO class index to target label
        class_map = {i: label for i, label in enumerate(target_labels)}

        # Check if any of the target labels are detected
        for i, label in enumerate(labels):
            if class_map[label] in target_labels and scores[i] >= threshold:
                score = scores[i]
                box = boxes[i]

                # Get the coordinates and dimensions of the detected object
                x1, y1, x2, y2 = box

                # Open the original image
                im = Image.open(image_path)

                # Crop the image using the detected coordinates
                im_crop = im.crop((x1, y1, x2, y2))

                # Convert image mode to RGB if it's RGBA
                if im_crop.mode == 'RGBA':
                    im_crop = im_crop.convert('RGB')

                # Save the cropped image with ID
                crop_path = os.path.join(crop_dir, f'{image_id}.jpg')
                im_crop.save(crop_path)
                print(f"Cropped image saved as '{crop_path}'.")
                detected = True
                return image_id  # Return the image ID when successfully cropped

    if not detected:
        print(f"Can't detect any target objects in image ID '{image_id}'.")
        return None  # Return None when no objects detected

# Check if path is a file or directory
if os.path.isfile(path):
    image_id = get_image_id(path)
    detect_and_crop(path, image_id)
elif os.path.isdir(path):
    # Loop through all files in the directory
    for idx, file_name in enumerate(sorted(os.listdir(path))):
        file_path = os.path.join(path, file_name)
        if os.path.isfile(file_path):
            image_id = f"image{idx + 1}"
            detect_and_crop(file_path, image_id)
else:
    print(f"Invalid path: {path}")

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

# เชื่อมต่อกับ Qdrant instance บนคลาวด์
qdrant_client = QdrantClient(
    url="https://8366dca9-9b40-481c-9a7c-102b62b118c2.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key="Ft3t6L99uoTsJW8IOk5VH2byKU-BQYqiieuxXFBDp99wo75od0ddAw",
)

# ชื่อ collection
collection_name = "vector_drug"

# โหลดโมเดล ResNet50 ที่ใช้ในการสร้าง embedding
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

def image_embedding(image_path):
    img = Image.open(image_path).convert('RGB').resize((640, 640))  # ปรับขนาดภาพให้เหมาะสม
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    return preds[0]  # คืนค่า embedding เป็น numpy array

def display_image(image_path):
    img = Image.open(image_path)
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'Query Image: {os.path.basename(image_path)}')  # แสดงชื่อไฟล์
    plt.show()

def search_similar_items(query_image_path, client, collection_name):
    # สร้าง embedding สำหรับรูปภาพ query
    query_embedding = image_embedding(query_image_path)

    # แสดงรูปภาพ query พร้อมชื่อไฟล์
    display_image(query_image_path)

    # ค้นหาคล้ายคลึงใน Qdrant
    search_result = client.search(
        collection_name=collection_name,
        query_vector=query_embedding.tolist(),
        limit=100  # ค้นหา 100 ผลลัพธ์เพื่อลดความเป็นไปได้ของการขาดข้อมูล
    )

    # เก็บคะแนนสูงสุดสำหรับแต่ละ class_name
    class_scores = {}
    for result in search_result:
        point_id = result.id
        score = result.score
        payload = result.payload
        class_name = payload.get("class", "Unknown")

        # อัปเดตคะแนนสูงสุดสำหรับ class_name นี้
        if class_name not in class_scores or score > class_scores[class_name]:
            class_scores[class_name] = score

    # เลือก 5 ผลลัพธ์ที่มีคะแนนสูงสุด โดยไม่ซ้ำกัน
    sorted_class_scores = sorted(class_scores.items(), key=lambda x: x[1], reverse=True)
    top_classes = sorted_class_scores[:5]  # เลือก 5 คลาสที่มีคะแนนสูงสุด

    # แสดงผลลัพธ์การค้นหา
    print("Search Results:")
    for class_name, score in top_classes:
        print(f"Class Name: {class_name}, Score: {score}")

def search_images_from_folder(folder_path, client, collection_name):
    # ตรวจสอบว่ามีไฟล์ในโฟลเดอร์หรือไม่
    if not os.listdir(folder_path):
        print(f"No images found in folder '{folder_path}'")
        return

    # ค้นหาและแสดงผลลัพธ์สำหรับแต่ละภาพในโฟลเดอร์
    for image_file in os.listdir(folder_path):
        if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, image_file)
            print(f"Searching for image: {image_path}")
            search_similar_items(image_path, client, collection_name)

# ตัวอย่างการใช้งาน
folder_path = '/content/drive/MyDrive/drug/CorpTest'
search_images_from_folder(folder_path, qdrant_client, collection_name)

ผิดไป 3class